In [1]:
import collections
collections.MutableMapping = collections.abc.MutableMapping
collections.MutableSequence = collections.abc.MutableSequence
collections.MutableSet = collections.abc.MutableSet

In [2]:
import collections
import time
import threading

In [3]:
from kortex_api.autogen.client_stubs.ActuatorConfigClientRpc import ActuatorConfigClient
from kortex_api.autogen.client_stubs.BaseClientRpc import BaseClient
from kortex_api.autogen.client_stubs.BaseCyclicClientRpc import BaseCyclicClient
from kortex_api.autogen.messages import Base_pb2, BaseCyclic_pb2, ActuatorConfig_pb2

from kortex_api.TCPTransport import TCPTransport
from kortex_api.UDPTransport import UDPTransport

from kortex_api.RouterClient import RouterClient, RouterClientSendOptions
from kortex_api.SessionManager import SessionManager
from kortex_api.autogen.messages import Session_pb2

In [4]:
import sys
sys.path.append('/home/vertix/Documents/positronic/')

In [5]:
import geom
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import numpy as np
# from pytorch3d.transforms import quaternion_invert, quaternion_multiply, matrix_to_quaternion, quaternion_to_matrix
# from pytorch3d.transforms import axis_angle_to_matrix, matrix_to_quaternion

In [9]:
transport = TCPTransport()
router = RouterClient(transport, RouterClient.basicErrorCallback)
transport.connect('192.168.1.10', 10000)

session_info = Session_pb2.CreateSessionInfo()
session_info.username = "admin"
session_info.password = "admin"
session_info.session_inactivity_timeout = 1000 * 1000   # (milliseconds)
session_info.connection_inactivity_timeout = 1000 * 1000 # (milliseconds)

sessionManager = SessionManager(router)
sessionManager.CreateSession(session_info)

base = BaseClient(router)

In [10]:
udp_transport = UDPTransport()
upd_router = RouterClient(udp_transport, RouterClient.basicErrorCallback)
udp_transport.connect('192.168.1.10', 10001)

udp_session = SessionManager(upd_router)
udp_session.CreateSession(session_info)
base_cyclic = BaseCyclicClient(upd_router)

In [11]:
class Kinova:
    def __init__(self, base, base_cyclic):
        self.base = base
        self.base_cyclic = base_cyclic
        self.base.ClearFaults()

    def reset(self):
        base_servo_mode = Base_pb2.ServoingModeInformation()
        base_servo_mode.servoing_mode = Base_pb2.SINGLE_LEVEL_SERVOING
        self.base.SetServoingMode(base_servo_mode)

        action_type = Base_pb2.RequestedActionType()
        action_type.action_type = Base_pb2.REACH_JOINT_ANGLES
        action_list = base.ReadAllActions(action_type)
        action_handle = None
        for action in action_list.action_list:
            if action.name == "Home":
                action_handle = action.handle

        base.ExecuteActionFromReference(action_handle)

    @property
    def joint_angles(self):
        feedback = self.base_cyclic.RefreshFeedback()
        return np.array([np.radians(a.position) for a in feedback.actuators])

    @property
    def position(self):
        feedback = self.base_cyclic.RefreshFeedback()
        return geom.Transform3D(
            np.array([feedback.base.tool_pose_x, feedback.base.tool_pose_y, feedback.base.tool_pose_z]),
            geom.Rotation.from_euler(
                [feedback.base.tool_pose_theta_x, feedback.base.tool_pose_theta_y, feedback.base.tool_pose_theta_z]))

    def set_target_position(self, position):
        pass

In [12]:
kinova = Kinova(base, base_cyclic)

In [13]:
kinova.reset()

In [29]:
kinova.position

Transform3D(t=[ 0.457 -0.006  0.435], q=[ 0.391  0.854 -0.102  0.329])

---

In [8]:
def robot_fk(angles):
    angles = geom.radians_to_degrees(angles)
    angles_pb = Base_pb2.JointAngles()
    for i, angle in enumerate(angles):
        angles_pb.joint_angles.add(joint_identifier=i, value=angle)
    try:
        fk_out = base.ComputeForwardKinematics(angles_pb)
    except:
        print(angles)
        raise
    return geom.Transform3D(translation=np.array([fk_out.x, fk_out.y, fk_out.z]),
                            quaternion=geom.Quaternion.from_euler([fk_out.theta_x, fk_out.theta_y, fk_out.theta_z]))

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
max_position = geom.degrees_to_radians(np.array([180, 128.9, 180, 147.8, 180, 120.3, 180]))
joint_angles_set = []
target_set = []
while len(joint_angles_set) < 3200 * 1:
    angles = np.random.uniform(-max_position, max_position, size=(7))
    try:
        target_set.append(robot_fk(angles))
        joint_angles_set.append(angles)
    except:
        continue
joint_angles_set = np.array(joint_angles_set)
joint_angles_set[:, 0] = -joint_angles_set[:, 0]

[-104.44380733   51.20547109 -164.02341569   98.00041812  -57.98826489
  -87.75071955  170.94020015]
[  4.31584539 -15.28273114  -6.92080085 -85.24894559 106.43111907
 111.26756796 -65.73425143]
[-118.39674101 -122.98797447 -116.75283472   91.21565432  133.79464857
  -64.45777269  113.7114917 ]
[-143.57968399    8.07253243 -142.67242919   50.23381637 -128.77935208
   54.65040603  -67.91732733]
[-173.51054699  -72.99829991  -17.84736523  -30.11004015  -95.67674233
  -90.14948521   96.8402083 ]
[-169.28964968 -113.02636123 -151.46328342  142.6945743   109.9716173
  -16.82859809  130.02563733]
[-140.03192264  -75.42778785 -117.70357691  105.71644352   79.62889203
  101.6302304    29.39805147]
[-98.14680345 -39.40304244 -42.66036865  85.36046725 157.12840189
  98.09695008  99.38224919]
[-158.95460051  -57.89359495  -31.55875649  -54.2339519    82.32593378
   26.44691337  -95.09775556]
[134.84521874  17.60690786  73.89118325 -71.65718764 125.65461461
  86.89391925 131.79466052]
[-148.084438

----
### Common functions

In [11]:
def closest_rotation_matrix(matrix):
    U, _, Vt = np.linalg.svd(matrix)
    R = np.dot(U, Vt)
    # det = np.linalg.det(R)
    # Vt[2, :] *= det
    # R = np.dot(U, Vt)
    return R

In [12]:
def geodesic_loss(predicted_rotations, true_rotations):
    batch_size = predicted_rotations.shape[0]
    identity = torch.eye(3, device=predicted_rotations.device).unsqueeze(0).repeat(batch_size, 1, 1)

    relative_rotation = torch.bmm(true_rotations.transpose(1, 2), predicted_rotations)
    trace = torch.diagonal(relative_rotation, dim1=-2, dim2=-1).sum(-1)
    geodesic_distance = torch.acos(torch.clamp((trace - 1) / 2, -1.0, 1.0))
    return torch.mean(geodesic_distance)

def orthogonality_loss(params):
    loss = 0
    for i in range(8):
        R = params[i, :3, :3]
        I = torch.eye(3, device=params.device)
        loss += torch.norm(torch.mm(R.t(), R) - I)  # Orthogonality loss
        loss += torch.abs(torch.det(R) - 1)  # Determinant loss
    return loss / 8

def translation_loss(predicted_positions, true_positions):
    return nn.MSELoss()(predicted_positions, true_positions)

def compute_loss(predicted_positions, true_positions, predicted_rotations, true_rotations, params,
                 or_weight=0.1, reg_weight=0.1):
    position_loss = translation_loss(predicted_positions, true_positions)
    orientation_loss = geodesic_loss(predicted_rotations, true_rotations)
    regularization_loss = orthogonality_loss(params)
    return position_loss + or_weight * orientation_loss + reg_weight * regularization_loss

def sqrt_translation_loss(predicted_positions, true_positions, _1, _2, params):
    return torch.sqrt(translation_loss(predicted_positions, true_positions))

In [13]:
def training_loop(joint_angles_set, target_set, params,
                  forward_kinematics_fn, loss_fn,
                  val_losses = {'Val Tr': sqrt_translation_loss,
                                'Val Rot': lambda _1, _2, p, t, _3:  geodesic_loss(p, t) / np.pi * 180,
                                'Var Reg': lambda _1, _2, _3, _4, pms: orthogonality_loss(pms)},
                  num_epochs=100, batch_size=32, lr=0.01):
    joint_angles = torch.tensor(joint_angles_set, dtype=torch.float32).to(device)
    end_effector_positions = torch.tensor([t.translation for t in target_set], dtype=torch.float32).to(device)
    end_effector_orientations = torch.tensor([t.quaternion.as_rotation_matrix for t in target_set], dtype=torch.float32).to(device)

    train_size = int(0.8 * len(joint_angles))

    train_joint_angles = joint_angles[:train_size]
    val_joint_angles = joint_angles[train_size:]

    train_end_effector_positions = end_effector_positions[:train_size]
    val_end_effector_positions = end_effector_positions[train_size:]
    train_end_effector_orientations = end_effector_orientations[:train_size]
    val_end_effector_orientations = end_effector_orientations[train_size:]

    optimizer = optim.Adam([params], lr=lr)

    num_batches = train_size // batch_size
    train_loss = torch.tensor(0.0, device=device)

    for epoch in range(num_epochs):
        permutation = torch.randperm(train_size)
        train_joint_angles = train_joint_angles[permutation]
        train_end_effector_positions = train_end_effector_positions[permutation]
        train_end_effector_orientations = train_end_effector_orientations[permutation]

        if epoch % 10 == 0:
            with torch.no_grad():
                T_val = forward_kinematics_fn(val_joint_angles, params)
                predicted_val_positions = T_val[:, :3, 3]
                predicted_val_orientations = T_val[:, :3, :3]

                val_result = {}
                for val_loss_name, val_loss_fn in val_losses.items():
                    val_loss = val_loss_fn(predicted_val_positions, val_end_effector_positions,
                                           predicted_val_orientations, val_end_effector_orientations, params)
                    val_result[val_loss_name] = val_loss.item()
            tl = np.sqrt(train_loss.item() / num_batches)
            report = f'Epoch [{epoch}/{num_epochs}], Train L: {tl:.7f}, '
            for val_loss_name, val_loss in val_result.items():
                report += f'{val_loss_name}: {val_loss:.7f}, '
            print(report)

        optimizer.zero_grad()
        train_loss = 0
        for batch_idx in range(num_batches):
            start_idx = batch_idx * batch_size
            end_idx = start_idx + batch_size

            joint_angles_batch = train_joint_angles[start_idx:end_idx]
            end_effector_positions_batch = train_end_effector_positions[start_idx:end_idx]
            end_effector_orientations_batch = train_end_effector_orientations[start_idx:end_idx]

            T = forward_kinematics_fn(joint_angles_batch, params)
            predicted_positions = T[:, :3, 3]
            predicted_orientations = T[:, :3, :3]

            batch_loss = loss_fn(predicted_positions, end_effector_positions_batch,
                                 predicted_orientations, end_effector_orientations_batch, params)
            batch_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_loss += batch_loss
        train_loss /= num_batches

---
### Transform matrix parametrisation

In [12]:
def transformation_matrix_3x4(params):
    batch_size = params.shape[0]
    T = torch.eye(4, device=params.device).unsqueeze(0).repeat(batch_size, 1, 1)
    T[:, :3, :4] = params
    return T

def rotation_matrix_around_z(theta):
    batch_size = theta.shape[0]
    R = torch.eye(4, device=theta.device).unsqueeze(0).repeat(batch_size, 1, 1)
    R[:, 0, 0] = torch.cos(theta)
    R[:, 0, 1] = -torch.sin(theta)
    R[:, 1, 0] = torch.sin(theta)
    R[:, 1, 1] = torch.cos(theta)
    return R

def forward_kinematics_3x4(joint_angles, params):
    batch_size = joint_angles.shape[0]
    T = torch.eye(4, device=joint_angles.device).unsqueeze(0).repeat(batch_size, 1, 1)
    for i in range(8):
        if i < 7:
            R_joint = rotation_matrix_around_z(joint_angles[:, i])
            T_joint = transformation_matrix_3x4(params[i].unsqueeze(0).repeat(batch_size, 1, 1))
            T_joint = torch.bmm(T_joint, R_joint)
        else:
            T_joint = transformation_matrix_3x4(params[i].unsqueeze(0).repeat(batch_size, 1, 1))
        T = torch.bmm(T, T_joint)
    return T

In [269]:
# region Transform matrix params initialisation
clean_initialisation = True
if clean_initialisation:
    params = torch.eye(3, 4, device=device).unsqueeze(0).repeat(8, 1, 1)
    params += 0.01 * torch.randn_like(params)  # Add small random perturbations
    params = params.to(device).requires_grad_(True)
else:
    params = torch.tensor([
        [[0.95999, 0.25863, 0.00000, 0.00000],
        [0.25983, -0.96103, 0.00000, 0.00000],
        [0.00000, 0.00000, 1.00000, 0.17415]],

        [[0.96725, 0.00000, -0.25973, -0.05303],
        [0.26238, 0.00000, 0.96725, 0.20503],
        [0.00000, -1.00155, 0.00000, 0.10893]],

        [[0.97877, 0.19192, 0.00000, 0.00000],
        [0.00000, 0.00000, -0.99765, -0.34695],
        [-0.19310, 0.97878, 0.00000, -0.20027]],

        [[0.90814, 0.36822, -0.20204, 0.00000],
        [0.19684, 0.05537, 0.97951, 0.02780],
        [0.37122, -0.92883, 0.00000, 0.07927]],

        [[0.84509, 0.35974, 0.39030, 0.13681],
        [0.35497, 0.16276, -0.91869, -0.34235],
        [-0.39467, 0.91681, 0.00000, -0.01446]],

        [[0.94290, 0.18952, -0.27561, -0.07539],
        [0.28330, 0.00000, 0.95991, 0.25185],
        [0.17853, -0.98264, 0.00000, -0.06802]],

        [[0.99947, 0.00000, 0.00000, 0.02646],
        [0.00000, 0.99947, 0.00000, -0.16470],
        [0.00000, 0.00000, 0.99947, -0.15307]],

        [[1.00000, 0.00000, 0.00000, 0.00000],
        [0.00000, 1.00000, 0.00000, 0.00000],
        [0.00000, 0.00000, 1.00000, -0.10911]]
    ], device=device, requires_grad=True)
# endregion initialisation

In [270]:
# Val Tr: 0.0013704, Var Reg: 0.1003958, reg = 0.01
# Val Tr: 0.0015980, Var Reg: 0.0753762, reg = 0.1
# Val Tr: 0.0013968, Var Reg: 0.0503261, reg = 0.1 after reg=1, (good_params)
training_loop(joint_angles_set, target_set, params, forward_kinematics_3x4,
              lambda *args, **kwargs: compute_loss(*args, **kwargs, or_weight=10, reg_weight=0.01),
               num_epochs=100, batch_size=32, lr=0.001)

Epoch [0/100], Train L: 0.0000000, Val Tr: 0.4347759, Val Rot: 128.1339569, Var Reg: 0.0591714, 
Epoch [10/100], Train L: 0.5129604, Val Tr: 0.3042877, Val Rot: 120.9816513, Var Reg: 0.8561605, 
Epoch [20/100], Train L: 0.5122877, Val Tr: 0.2932837, Val Rot: 120.8232193, Var Reg: 1.0485711, 
Epoch [30/100], Train L: 0.5117086, Val Tr: 0.2813696, Val Rot: 121.1250839, Var Reg: 1.1517088, 
Epoch [40/100], Train L: 0.5112205, Val Tr: 0.2758288, Val Rot: 121.3677063, Var Reg: 1.2857041, 
Epoch [50/100], Train L: 0.5108935, Val Tr: 0.2728305, Val Rot: 121.5100861, Var Reg: 1.3919470, 
Epoch [60/100], Train L: 0.5106660, Val Tr: 0.2710274, Val Rot: 121.7244415, Var Reg: 1.4573823, 
Epoch [70/100], Train L: 0.5104812, Val Tr: 0.2718749, Val Rot: 121.8785019, Var Reg: 1.5109535, 
Epoch [80/100], Train L: 0.5103269, Val Tr: 0.2727478, Val Rot: 122.0951309, Var Reg: 1.5403166, 
Epoch [90/100], Train L: 0.5101508, Val Tr: 0.2730571, Val Rot: 122.2616425, Var Reg: 1.5842466, 


In [244]:
for p in params:
    print(np.linalg.det(p[:3, :3].cpu().detach().numpy()))

0.99658805
0.9975116
0.9953971
0.99630344
0.99839497
0.9951809
0.9961921
0.9987566


In [148]:
closest_rotation_matrix(params[0, :3, :3].cpu().detach().numpy()), params[0, :3, :3].cpu().detach().numpy()

(array([[-0.84363806,  0.53667235,  0.01604977],
        [ 0.536898  ,  0.84345704,  0.01791053],
        [ 0.00392521, -0.0237271 ,  0.99971074]], dtype=float32),
 array([[-0.14482734,  0.09030467,  0.00056035],
        [ 0.09195863,  0.1414392 ,  0.00585278],
        [ 0.00390666, -0.00212199,  0.14228664]], dtype=float32))

In [138]:
good_params = params.clone()

In [20]:
p_rot_matrix = params[0, :3, :3] @ params[0, :3, :3].t()
print(np.array2string(p_rot_matrix.cpu().detach().numpy(), formatter={'float_kind':lambda x: "%.5f" % x}))

[[0.99985 -0.00003 -0.00007]
 [-0.00003 0.99988 0.00014]
 [-0.00007 0.00014 1.00011]]


In [21]:
_fk_res = forward_kinematics_3x4(torch.tensor([[0, 0, 0, np.pi / 3, 0, 0, 0]]).to(device), params)
geom.Transform3D(translation=_fk_res[0, :3, 3].cpu().detach().numpy(),
                 quaternion=matrix_to_quaternion(_fk_res[0, :3, :3]).cpu().detach().numpy())
# print(np.array2string(_fk_res[0].cpu().detach().numpy(), formatter={'float_kind':lambda x: "%.4f" % x}))

Transform3D(translation=array([ 0.41683146, -0.02172742,  0.9507421 ], dtype=float32), quaternion=array([ 0.03985714, -0.6964763 , -0.165107  ,  0.04246614], dtype=float32))

In [22]:
_fk_res = forward_kinematics_3x4(torch.tensor([[0, 0, 0, 0, 0, 0, 0]]).to(device), params)
geom.Transform3D(translation=_fk_res[0, :3, 3].cpu().detach().numpy(),
                 quaternion=matrix_to_quaternion(_fk_res[0, :3, :3]).cpu().detach().numpy())
# print(np.array2string(_fk_res[0].cpu().detach().numpy(), formatter={'float_kind':lambda x: "%.4f" % x}))

Transform3D(translation=array([-0.00465913, -0.01732557,  1.1907881 ], dtype=float32), quaternion=array([ 0.00453524, -0.7070728 ,  0.01540003, -0.00438324], dtype=float32))

In [23]:
robot_fk(np.array([0, 0, 0, np.pi / 3, 0, 0, 0]))

Transform3D(translation=array([ 0.41688028, -0.0220619 ,  0.95082903]), quaternion=Quaternion([-0.89972798,  0.32380798, -0.1182735 ,  0.26767391]))

In [24]:
robot_fk(np.array([0, 0, 0, 0, 0, 0, 0]))

Transform3D(translation=array([-0.00454275, -0.01758629,  1.19172156]), quaternion=Quaternion([ 0.76933339,  0.04427104, -0.28721488,  0.56892339]))

In [25]:
print(np.array2string(params.cpu().detach().numpy(), formatter={'float_kind':lambda x: "%.5f" % x}))

[[[0.77013 -0.63777 0.00152 -0.00038]
  [-0.63779 -0.77013 -0.00339 0.00016]
  [0.00319 0.00158 -1.00005 0.14570]]

 [[-0.50354 -0.58628 0.63470 -0.09682]
  [0.41384 0.48130 0.77278 -0.11863]
  [0.75822 -0.65168 -0.00027 -0.13760]]

 [[0.38109 -0.53141 0.75626 -0.24756]
  [0.43695 -0.61764 -0.65384 0.21475]
  [-0.81469 -0.57976 0.00314 0.16431]]

 [[0.49512 0.30655 0.81292 0.31243]
  [-0.69016 -0.42956 0.58226 0.22499]
  [0.52771 -0.84953 -0.00111 -0.09528]]

 [[0.73179 -0.42949 0.52895 -0.19097]
  [0.45553 -0.26895 -0.84859 0.30669]
  [0.50668 0.86234 -0.00124 -0.39649]]

 [[-0.49508 -0.68594 0.53346 0.14786]
  [0.30728 0.43617 0.84588 0.23362]
  [-0.81285 0.58265 -0.00514 0.04368]]

 [[-0.70391 0.08640 -0.70507 0.09646]
  [0.11995 -0.96385 -0.23807 -0.11244]
  [-0.70013 -0.25233 0.66808 -0.23853]]

 [[-0.66671 0.33323 -0.66645 -0.00077]
  [-0.33348 0.66649 0.66668 -0.00009]
  [-0.66649 -0.66684 0.33322 -0.05638]]]


---
### DH Parametrisation

In [26]:
# Define the transformation matrix for a single joint using DH parameters
def transformation_matrix_dh(theta, d, a, alpha):
    if theta.dim() == 0:  # Scalar input
        theta = theta.unsqueeze(0)
    batch_size = theta.shape[0]
    T = torch.eye(4, device=theta.device).unsqueeze(0).repeat(batch_size, 1, 1)
    T[:, 0, 0] = torch.cos(theta)
    T[:, 0, 1] = -torch.sin(theta) * torch.cos(alpha)
    T[:, 0, 2] = torch.sin(theta) * torch.sin(alpha)
    T[:, 0, 3] = a * torch.cos(theta)
    T[:, 1, 0] = torch.sin(theta)
    T[:, 1, 1] = torch.cos(theta) * torch.cos(alpha)
    T[:, 1, 2] = -torch.cos(theta) * torch.sin(alpha)
    T[:, 1, 3] = a * torch.sin(theta)
    T[:, 2, 1] = torch.sin(alpha)
    T[:, 2, 2] = torch.cos(alpha)
    T[:, 2, 3] = d
    return T

def forward_kinematics_dh(joint_angles, dh_params):
    batch_size = joint_angles.shape[0]
    T = torch.eye(4, device=joint_angles.device).unsqueeze(0).repeat(batch_size, 1, 1)
    for i in range(8):
        theta, d, a, alpha = dh_params[i]
        if i < 7:
            T_joint = transformation_matrix_dh(joint_angles[:, i] + theta, d, a, alpha)
        else:
            T_joint = transformation_matrix_dh(theta.repeat(batch_size), d, a, alpha)
        T = torch.bmm(T, T_joint)
    return T

In [42]:
def extract_dh_parameters(T):
    # Extract rotation matrix and position vector
    R = T[:3, :3]
    p = T[:3, 3]
    theta = np.arctan2(R[1, 0], R[0, 0])
    d = p[2]
    a = np.sqrt(p[0]**2 + p[1]**2)
    alpha = np.arctan2(R[2, 1], R[2, 2])

    return theta, d, a, alpha

# Assuming `params` is the learned transformation matrix parameters
learned_params = params.cpu().detach().numpy()

# Extract DH parameters from the learned transformation matrices
dh_params_list = []
for i in range(learned_params.shape[0]):
    T = learned_params[i]
    theta, d, a, alpha = extract_dh_parameters(T)
    dh_params_list.append([theta, d, a, alpha])

# Convert the list to a tensor and move to the appropriate device
dh_params = torch.tensor(dh_params_list, device=device, requires_grad=True)

In [183]:
if clean_initialisation:
    dh_params = torch.zeros(8, 4, device=device, requires_grad=True)
else:
    dh_params = torch.tensor([
        [0.0, 0.15643, 0.0, 3.1416],  # Joint 1
        [0.0, -0.12838, 0.005375, 1.5708],  # Joint 2
        [0.0, -0.006375, -0.21038, -1.5708],  # Joint 3
        [0.0, -0.21038, 0.006375, 1.5708],  # Joint 4
        [0.0, -0.006375, -0.20843, -1.5708],  # Joint 5
        [0.0, -0.10593, 0.00017505, 1.5708],  # Joint 6
        [0.0, -0.00017505, -0.10593, -1.5708],  # Joint 7
        [0.0, -0.061525, 0.0, 3.1416]  # End Effector
    ], requires_grad=True, device=device)

In [35]:
def dh_loss(predicted_positions, true_positions, predicted_rotations, true_rotations, params, reg_weight=0.01):
    return translation_loss(predicted_positions, true_positions)

In [185]:
training_loop(joint_angles_set, target_set, dh_params, forward_kinematics_dh, dh_loss,
               val_losses = {'Val Tr': sqrt_translation_loss},
               num_epochs=100, batch_size=32, lr=0.001)

Epoch [0/100], Train L: 0.0000000, Val Tr: 0.2266131, 
Epoch [10/100], Train L: 0.0249289, Val Tr: 0.2266761, 
Epoch [20/100], Train L: 0.0249321, Val Tr: 0.2270195, 
Epoch [30/100], Train L: 0.0249221, Val Tr: 0.2267473, 
Epoch [40/100], Train L: 0.0249253, Val Tr: 0.2266841, 
Epoch [50/100], Train L: 0.0249289, Val Tr: 0.2265368, 
Epoch [60/100], Train L: 0.0249128, Val Tr: 0.2264510, 
Epoch [70/100], Train L: 0.0249268, Val Tr: 0.2267034, 
Epoch [80/100], Train L: 0.0249167, Val Tr: 0.2266383, 
Epoch [90/100], Train L: 0.0249227, Val Tr: 0.2270355, 


----
### Translation and quaternion parametrisation

In [195]:
def transformation_matrix_tq(params):
    batch_size = params.shape[0]
    T = torch.eye(4, device=params.device).unsqueeze(0).repeat(batch_size, 1, 1)
    T[:, :3, :3] = quaternion_to_matrix(params[..., :4] / params[..., :4].norm(dim=1, keepdim=True))
    T[:, :3, 3] = params[..., 4:7]
    return T

def forward_kinematics_tq(joint_angles, params):
    batch_size = joint_angles.shape[0]
    T = torch.eye(4, device=joint_angles.device).unsqueeze(0).repeat(batch_size, 1, 1)
    for i in range(8):
        if i < 7:
            R_joint = rotation_matrix_around_z(joint_angles[:, i])
            T_joint = transformation_matrix_tq(params[i].repeat(batch_size, 1))
            T_joint = torch.bmm(T_joint, R_joint)
        else:
            T_joint = transformation_matrix_tq(params[i].repeat(batch_size, 1))
        T = torch.bmm(T, T_joint)
    return T

In [186]:
params_tq = nn.Parameter(torch.randn(8, 7, device=device))
params_tq.data[:, :4] = params_tq.data[:, :4] / params_tq.data[:, :4].norm(dim=1, keepdim=True)
params_tq.data[:, 4:] = 0.01 * torch.randn(8, 3, device=device)

In [196]:
params_tq = nn.Parameter(torch.randn(8, 7, device=device))
for i, p in enumerate(params):
    params_tq[i, :4].data.copy_(matrix_to_quaternion(p[:3, :3]).data)
    params_tq[i, 4:].data.copy_(p[:3, 3].data)
params_tq.data[:, :4] = params_tq.data[:, :4] / params_tq.data[:, :4].norm(dim=1, keepdim=True)

In [197]:
training_loop(joint_angles_set, target_set, params_tq, forward_kinematics_tq, dh_loss,
               val_losses = {'Val Tr': sqrt_translation_loss},
               num_epochs=100, batch_size=32, lr=0.001)

Epoch [0/100], Train L: 0.0000000, Val Tr: 0.0047930, 
Epoch [10/100], Train L: 0.0001802, Val Tr: 0.0017436, 
Epoch [20/100], Train L: 0.0001699, Val Tr: 0.0015785, 
Epoch [30/100], Train L: 0.0001679, Val Tr: 0.0014288, 
Epoch [40/100], Train L: 0.0001713, Val Tr: 0.0016356, 
Epoch [50/100], Train L: 0.0001674, Val Tr: 0.0015507, 
Epoch [60/100], Train L: 0.0001750, Val Tr: 0.0015528, 
Epoch [70/100], Train L: 0.0001765, Val Tr: 0.0015839, 
Epoch [80/100], Train L: 0.0001736, Val Tr: 0.0015739, 
Epoch [90/100], Train L: 0.0001697, Val Tr: 0.0014529, 


In [218]:
CHAIN = []
for p in params_tq:
    t = p[4:].cpu().detach().numpy()
    q = p[:4].cpu().detach().numpy()
    q = q / np.linalg.norm(q)
    # print(f'Transform3D({t.tolist()}, Quaternion({q.tolist()})),')
    CHAIN.append(geom.Transform3D(translation=t, quaternion=geom.Quaternion(*q)))

In [ ]:
def __fk(joints, CHAIN):
    joints = degrees_to_radians(joints)
    joints[0] = -joints[0]  # This is on purpose, the first joint is inverted

    result = Transform3D()
    for i in range(8):
        T_joint = _KINOVA_CHAIN[i]
        if i < 7:
            R_joint = Transform3D(quaternion=Quaternion.from_euler([0, 0, joints[i]]))
            T_joint = T_joint * R_joint
        result = result * T_joint
    return result

In [229]:
from hardware import kinova
kinova._forward_kinematics(np.array([np.pi / 4, 0, 0, np.pi / 3, 0, 0, 0]))

Transform3D(translation=Quaternion([ 0.27638114, -0.31061917,  0.94938417]), quaternion=Quaternion([ 0.6885704 , -0.32237743,  0.61590285,  0.20641523]))

In [234]:
for ja, t in zip(joint_angles_set[:10], target_set[:10]):
    fk = kinova._forward_kinematics(ja)
    print(fk.translation - t.translation)
    # print(fk.quaternion.inv * t.quaternion)

[0.00039361 0.00118508 0.00093426]
[ 0.00050046  0.0001458  -0.00222602]
[-2.10653209e-04 -9.50002942e-05 -2.81321447e-04]
[-8.52860448e-05  4.45588195e-04  1.17181741e-03]
[ 1.23069345e-03  1.32805648e-04 -5.92699297e-05]
[ 0.00040665 -0.00016971 -0.00085031]
[0.00060344 0.00041321 0.00127315]
[0.00151878 0.0019804  0.00334766]
[0.00248696 0.00143497 0.00087649]
[-0.00064614  0.00062047 -0.00117341]


In [227]:
forward_kinematics_tq(torch.tensor([[np.pi / 4, 0, 0, np.pi / 3, 0, 0, 0]]).to(device), params_tq)[0, :3, :3]

tensor([[ 0.1109, -0.7073, -0.6982],
        [ 0.1523, -0.6821,  0.7152],
        [-0.9821, -0.1856,  0.0321]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [221]:
robot_fk(np.array([np.pi / 4, 0, 0, np.pi / 3, 0, 0, 0]))

Transform3D(translation=array([ 0.27664271, -0.3105503 ,  0.95157391]), quaternion=Quaternion([-0.7116911 ,  0.44604303, -0.51713448, -0.16466126]))

In [258]:
base.ComputeForwardKinematics(base.GetMeasuredJointAngles())

x: 0.4568536
y: -0.005980493
z: 0.435407
theta_x: 90.21811
theta_y: -0.6986531
theta_z: 88.07178

In [264]:
[a.value for a in base.GetMeasuredJointAngles().joint_angles]

[0.00035041754017584026,
 15.00699234008789,
 179.99844360351562,
 229.9932403564453,
 359.99859619140625,
 54.99531936645508,
 89.9994888305664]

---
### Axis angle

In [104]:
def transformation_matrix_axis_angle(params):
    batch_size = params.shape[0]
    T = torch.eye(4, device=params.device).unsqueeze(0).repeat(batch_size, 1, 1)
    T[:, :3, :3] = axis_angle_to_matrix(params[..., :3])
    T[:, :3, 3] = params[..., 3:6]
    return T

def forward_kinematics_axis_angle(joint_angles, params):
    batch_size = joint_angles.shape[0]
    T = torch.eye(4, device=joint_angles.device).unsqueeze(0).repeat(batch_size, 1, 1)
    for i in range(8):
        if i < 7:
            R_joint = rotation_matrix_around_z(joint_angles[:, i])
            T_joint = transformation_matrix_axis_angle(params[i].repeat(batch_size, 1))
            T_joint = torch.bmm(T_joint, R_joint)
        else:
            T_joint = transformation_matrix_axis_angle(params[i].repeat(batch_size, 1))
        T = torch.bmm(T, T_joint)
    return T

In [105]:
def init_axis_angle_params():
    params = torch.zeros(8, 6, device=device)
    axis = torch.randn(8, 3, device=device)
    axis = axis / axis.norm(dim=1, keepdim=True)  # Normalize to unit vectors
    angle = torch.randn(8, 1, device=device) * 0.1  # Small random angles
    params[:, :3] = axis * angle
    params[:, 3:] = torch.randn(8, 3, device=device) * 0.1  # Small random translations
    return nn.Parameter(params)

params_axis_angle = init_axis_angle_params()

In [106]:
training_loop(joint_angles_set, target_set, params_axis_angle, forward_kinematics_axis_angle, dh_loss,
               val_losses = {'Val Tr': sqrt_translation_loss},
               num_epochs=100, batch_size=32, lr=0.001)

Epoch [0/100], Train L: 0.0000000, Val Tr: 0.5458503, 
Epoch [10/100], Train L: 0.0403375, Val Tr: 0.3598517, 
Epoch [20/100], Train L: 0.0403006, Val Tr: 0.3601849, 
Epoch [30/100], Train L: 0.0399200, Val Tr: 0.3565348, 
Epoch [40/100], Train L: 0.0392160, Val Tr: 0.3507843, 
Epoch [50/100], Train L: 0.0390921, Val Tr: 0.3493647, 
Epoch [60/100], Train L: 0.0390696, Val Tr: 0.3490727, 
Epoch [70/100], Train L: 0.0390726, Val Tr: 0.3492168, 
Epoch [80/100], Train L: 0.0390615, Val Tr: 0.3492782, 
Epoch [90/100], Train L: 0.0390498, Val Tr: 0.3486316, 


---
### Exp map

In [107]:
def exp_map_to_matrix(exp_map):
    theta = torch.norm(exp_map, dim=-1, keepdim=True)
    w = exp_map / (theta + 1e-8)
    wx = torch.zeros(exp_map.shape[0], 3, 3, device=exp_map.device)
    wx[:, 0, 1], wx[:, 0, 2], wx[:, 1, 2] = -w[:, 2], w[:, 1], -w[:, 0]
    wx[:, 1, 0], wx[:, 2, 0], wx[:, 2, 1] = w[:, 2], -w[:, 1], w[:, 0]
    I = torch.eye(3, device=exp_map.device).unsqueeze(0)
    R = I + torch.sin(theta).unsqueeze(-1) * wx + (1 - torch.cos(theta)).unsqueeze(-1) * torch.bmm(wx, wx)
    return R

def transformation_matrix_exp_map(params):
    batch_size = params.shape[0]
    T = torch.eye(4, device=params.device).unsqueeze(0).repeat(batch_size, 1, 1)
    T[:, :3, :3] = exp_map_to_matrix(params[..., :3])
    T[:, :3, 3] = params[..., 3:6]
    return T

def forward_kinematics_exp_map(joint_angles, params):
    batch_size = joint_angles.shape[0]
    T = torch.eye(4, device=joint_angles.device).unsqueeze(0).repeat(batch_size, 1, 1)
    for i in range(8):
        if i < 7:
            R_joint = rotation_matrix_around_z(joint_angles[:, i])
            T_joint = transformation_matrix_exp_map(params[i].repeat(batch_size, 1))
            T_joint = torch.bmm(T_joint, R_joint)
        else:
            T_joint = transformation_matrix_exp_map(params[i].repeat(batch_size, 1))
        T = torch.bmm(T, T_joint)
    return T

In [108]:
def init_exp_map_params():
    params = torch.zeros(8, 6, device=device)
    params[:, :3] = torch.randn(8, 3, device=device) * 0.1  # Small random exp map values
    params[:, 3:] = torch.randn(8, 3, device=device) * 0.1  # Small random translations
    return nn.Parameter(params)

params_exp_map = init_exp_map_params()

In [109]:
training_loop(joint_angles_set, target_set, params_exp_map, forward_kinematics_exp_map, dh_loss,
               val_losses = {'Val Tr': sqrt_translation_loss},
               num_epochs=100, batch_size=32, lr=0.001)

Epoch [0/100], Train L: 0.0000000, Val Tr: 0.4556128, 
Epoch [10/100], Train L: 0.0402838, Val Tr: 0.3595736, 
Epoch [20/100], Train L: 0.0401519, Val Tr: 0.3589512, 
Epoch [30/100], Train L: 0.0398864, Val Tr: 0.3565167, 
Epoch [40/100], Train L: 0.0394437, Val Tr: 0.3524919, 
Epoch [50/100], Train L: 0.0391771, Val Tr: 0.3501579, 
Epoch [60/100], Train L: 0.0390972, Val Tr: 0.3492624, 
Epoch [70/100], Train L: 0.0390804, Val Tr: 0.3492976, 
Epoch [80/100], Train L: 0.0390937, Val Tr: 0.3493089, 
Epoch [90/100], Train L: 0.0390619, Val Tr: 0.3491254, 


---
Dual quaternions

In [113]:
def dual_quaternion_to_matrix(dq):
    q_real, q_dual = dq[..., :4], dq[..., 4:]
    R = quaternion_to_matrix(q_real)
    t = 2 * quaternion_multiply(q_dual, quaternion_invert(q_real))[..., :3]
    return R, t

def transformation_matrix_dual_quaternion(params):
    batch_size = params.shape[0]
    T = torch.eye(4, device=params.device).unsqueeze(0).repeat(batch_size, 1, 1)
    R, t = dual_quaternion_to_matrix(params)
    T[:, :3, :3] = R
    T[:, :3, 3] = t
    return T

def forward_kinematics_dual_quaternion(joint_angles, params):
    batch_size = joint_angles.shape[0]
    T = torch.eye(4, device=joint_angles.device).unsqueeze(0).repeat(batch_size, 1, 1)
    for i in range(8):
        if i < 7:
            R_joint = rotation_matrix_around_z(joint_angles[:, i])
            T_joint = transformation_matrix_dual_quaternion(params[i].unsqueeze(0).repeat(batch_size, 1))
            T_joint = torch.bmm(T_joint, R_joint)
        else:
            T_joint = transformation_matrix_dual_quaternion(params[i].unsqueeze(0).repeat(batch_size, 1))
        T = torch.bmm(T, T_joint)
    return T

In [114]:
def init_dual_quaternion_params():
    params = torch.zeros(8, 8, device=device)
    # Real part (rotation)
    params[:, :4] = torch.randn(8, 4, device=device)
    params[:, :4] = params[:, :4] / params[:, :4].norm(dim=1, keepdim=True)  # Normalize
    # Dual part (translation)
    params[:, 4:7] = torch.randn(8, 3, device=device) * 0.1  # Small random translations
    params[:, 7] = 0  # Last component of dual part is typically 0
    return nn.Parameter(params)

params_dual_quaternion = init_dual_quaternion_params()

In [117]:
training_loop(joint_angles_set, target_set, params_dual_quaternion, forward_kinematics_dual_quaternion, dh_loss,
               val_losses = {'Val Tr': sqrt_translation_loss},
               num_epochs=100, batch_size=32, lr=0.001)

Epoch [0/100], Train L: 0.0000000, Val Tr: 0.3411884, 
Epoch [10/100], Train L: 0.0461037, Val Tr: 0.3638723, 
Epoch [20/100], Train L: 0.0382955, Val Tr: 0.3412696, 
Epoch [30/100], Train L: 0.0382836, Val Tr: 0.3421803, 
Epoch [40/100], Train L: 0.0382783, Val Tr: 0.3410841, 
Epoch [50/100], Train L: 0.0382734, Val Tr: 0.3415281, 
Epoch [60/100], Train L: 0.0404471, Val Tr: 0.3594979, 
Epoch [70/100], Train L: 0.0403403, Val Tr: 0.3603511, 
Epoch [80/100], Train L: 0.0403492, Val Tr: 0.3608832, 
Epoch [90/100], Train L: 0.0403230, Val Tr: 0.3608685, 


---
### Inverse kinematics

In [15]:
import geom
import importlib
importlib.reload(geom)

from hardware import kinova

In [23]:
_KINOVA_CHAIN = [
    geom.Transform3D([-7.732256199233234e-05, 0.0004820869071409106, 0.1384558230638504],
                geom.Quaternion(0.9999598860740662, -0.0014847038546577096, -0.0004586570430546999, 0.008817019872367382)),
    geom.Transform3D([-0.0020223343744874, -0.11016443371772766, 0.14530658721923828],
                geom.Quaternion(0.7064821720123291, -0.7075875997543335, 0.014157610014081001, -0.0015306948916986585)),
    geom.Transform3D([-0.012653934769332409, -0.6051681637763977, 0.0995088443160057],
                geom.Quaternion(0.7058353424072266, -0.7080992460250854, 0.019063977524638176, 0.005337915848940611)),
    geom.Transform3D([0.00476058479398489, 0.14438331127166748, 0.18294934928417206],
                geom.Quaternion(0.70661860704422, 0.7065088152885437, -0.03622652217745781, 0.014936398714780807)),
    geom.Transform3D([-0.031046129763126373, -0.4425973892211914, 0.13292159140110016],
                geom.Quaternion(0.7047972083091736, -0.7014123201370239, 0.09579798579216003, 0.045874472707509995)),
    geom.Transform3D([0.028989970684051514, 0.12411590665578842, 0.129145547747612],
                geom.Quaternion(0.703943133354187, 0.6839388608932495, -0.1895177811384201, 0.027831479907035828)),
    geom.Transform3D([-0.051960721611976624, -0.15933051705360413, 0.0792686715722084],
                geom.Quaternion(0.9999746084213257, 0.0043069422245025635, -0.0023099626414477825, 0.005186134018003941)),
    geom.Transform3D([0.00040886219358071685, -0.000173802807694301, 0.04850476235151291],
                geom.Quaternion(0.9999657869338989, 0.0035003339871764183, -0.0015276813646778464, 0.007339356932789087)),
]

In [18]:
test = joint_angles_set[0]
test[0] = -test[0]

In [55]:
test_degrees = geom.radians_to_degrees(test)
test_degrees

array([ 35.11296165,  18.35642287, 153.17966586,  95.4550201 ,
       103.89915645,  17.13910297,  45.79967953])

In [58]:
_inverse_kinematics(kinova._forward_kinematics(test_degrees), test_degrees + 20)

array([ 35.10227907,  18.40239598, 153.43369245,  95.44563768,
       103.9076728 ,  16.44754914,  46.24153973])

In [24]:
_forward_kinematics(test)

Transform3D(t=[-0.35   0.041  0.793], q=[-0.189  0.979 -0.019 -0.077])

In [57]:
from scipy.optimize import least_squares
def _inverse_kinematics(target, initial_position=None):
    if initial_position is None:
        initial_position = np.zeros(7)
    initial_position = np.array(initial_position, dtype=np.float64)
    target = target.as_matrix

    def objective_function(joint_angles):
        fk_result = kinova._forward_kinematics_matrix(joint_angles)
        position_error = fk_result[:3, 3] - target[:3, 3]
        orientation_error = 0.1 * (fk_result[:3, :3] - target[:3, :3])
        regularization = 0.0001 * geom.degrees_to_radians(np.linalg.norm(joint_angles - initial_position))
        return np.concatenate((position_error, orientation_error.flatten(), [regularization]))

    result = least_squares(objective_function, initial_position, method='lm')
    return result.x


In [21]:
def _forward_kinematics(joints):  # joints in radians
    joints = np.array(joints)
    joints[0] = -joints[0]  # This is on purpose, the first joint is inverted

    result = geom.Transform3D()
    for i in range(8):
        T_joint = _KINOVA_CHAIN[i]
        if i < 7:
            R_joint = geom.Transform3D(quaternion=geom.Quaternion.from_euler([0, 0, joints[i]]))
            T_joint = T_joint * R_joint
        result = result * T_joint
    return result


In [100]:
_forward_kinematics(np.array([np.pi / 4, 0, 0, np.pi / 3, 0, 0, 0]))

Transform3D(t=[ 0.276 -0.311  0.949], q=[ 0.689 -0.322  0.616  0.206])

In [154]:
from scipy.optimize import least_squares

def inverse_kinematics(target, initial_position=None):
    if initial_position is None:
        initial_position = np.zeros(7)
    initial_position = np.array(initial_position, dtype=np.float64)

    def objective_function(joint_angles):
        fk_result = _forward_kinematics(joint_angles)
        position_error = fk_result.translation - target.translation.astype(np.float64)
        orientation_error = 0.1 * (fk_result.quaternion.as_rotation_matrix - target.quaternion.as_rotation_matrix)
        regularization = 0.01 * np.linalg.norm(joint_angles)
        return np.concatenate((position_error, orientation_error.flatten(), [regularization]))

    result = least_squares(objective_function, initial_position, method='lm', ftol=1e-10)
    return result.x, result

In [155]:
res = None

In [156]:
res, _ = inverse_kinematics(target_set[0], initial_position=res)
res

array([ 5.57305105,  2.08830299, -9.49433737, -7.58956219,  4.10714708,
        3.72853255, -0.07694274])

In [157]:
_

     message: `ftol` termination condition is satisfied.
     success: True
      status: 2
         fun: [-6.176e-03 -2.678e-03 ... -1.197e-02  1.463e-01]
           x: [ 5.573e+00  2.088e+00 -9.494e+00 -7.590e+00  4.107e+00
                3.729e+00 -7.694e-02]
        cost: 0.010894393942519155
         jac: [[ 3.122e-01 -2.924e-01 ... -2.884e-02 -8.125e-05]
               [-2.338e-01 -2.514e-01 ...  1.284e-01 -2.305e-04]
               ...
               [-2.900e-04  7.566e-02 ... -8.515e-04 -3.794e-04]
               [ 3.810e-03  1.428e-03 ...  2.549e-03 -5.260e-05]]
        grad: [ 2.478e-09  1.805e-08 -5.050e-09 -4.392e-08 -5.060e-08
               -1.453e-08  8.316e-09]
  optimality: 5.0596153475492736e-08
 active_mask: [0 0 0 0 0 0 0]
        nfev: 200
        njev: None

In [158]:
clamp_joints(res)

array([-0.71013425,  2.08830299,  3.07203325, -1.30637689, -2.17603823,
       -2.55465276, -0.07694274])

In [159]:
res, _forward_kinematics(res), target_set[0]

(array([ 5.57305105,  2.08830299, -9.49433737, -7.58956219,  4.10714708,
         3.72853255, -0.07694274]),
 Transform3D(t=[ 0.234  0.312 -0.102], q=[ 0.228 -0.585 -0.195  0.753]),
 Transform3D(t=[ 0.24   0.315 -0.105], q=[-0.227  0.533  0.193 -0.792]))

[TCPTransport.send] ERROR: None
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/vertix/miniconda3/envs/positronic/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/vertix/miniconda3/envs/positronic/lib/python3.11/site-packages/kortex_api/SessionManager.py", line 125, in run
    self.parent.KeepAlive(options=options)
  File "/home/vertix/miniconda3/envs/positronic/lib/python3.11/site-packages/kortex_api/autogen/client_stubs/SessionClientRpc.py", line 59, in KeepAlive
    future = self.router.send(None, 1, SessionFunctionUid.uidKeepAlive, deviceId, options)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vertix/miniconda3/envs/positronic/lib/python3.11/site-packages/kortex_api/RouterClient.py", line 71, in send
    self.transport.send(payloadMsgFrame)
  File "/home/vertix/miniconda3/envs/positronic/lib/python3.11/site-packages/kortex_api/TCPTransport.py", line 8

[Errno 104] Connection reset by peer
[Errno 32] Broken pipe


In [41]:
target_set[0]

Transform3D(t=[ 0.24   0.315 -0.105], q=[-0.227  0.533  0.193 -0.792])

In [307]:
target_set[0]

Transform3D(translation=array([-0.23788337,  0.13711733,  1.13880122]), quaternion=Quaternion([-0.96724907,  0.18783113, -0.13580628, -0.10346672]))

In [64]:
def clamp_joints(joints):
    return np.mod(joints + np.pi, 2 * np.pi) - np.pi

In [71]:
initial_position = np.zeros(7)
target = target_set[0]

def objective_function(joint_angles):
    fk_result = _forward_kinematics(joint_angles)
    position_error = fk_result.translation - target.translation
    orientation_error = 0.1 * (fk_result.quaternion.as_rotation_matrix - target.quaternion.as_rotation_matrix)
    regularization = 0.1 * np.linalg.norm(joint_angles)
    return np.concatenate((position_error, orientation_error.flatten(), [regularization]))

result = least_squares(objective_function, initial_position, method='lm')

In [72]:
clamp_joints(result.x)

array([-0.42775674,  2.83370908,  0.84918635, -0.89696527, -0.15843906,
       -1.3713416 ,  0.01809427])

In [73]:
_forward_kinematics(clamp_joints(result.x)), target

(Transform3D(t=[ 0.285  0.393 -0.122], q=[ 0.323 -0.723 -0.053  0.608]),
 Transform3D(t=[ 0.24   0.315 -0.105], q=[-0.227  0.533  0.193 -0.792]))